In [1]:
# para qualquer partida, de qualquer competicao, a url deve ser
# https://www.cbf.com.br/futebol-brasileiro/competicoes/{competicao}/{temporada}/{jogo}?ref=linha#escalacao'

# desde 2012:
#  todas as edições do brasileirão têm 20 equipes e 380 jogos
#  o número de equipes e partidas tem variado na copa do brasil

In [46]:
import datetime
import json
import locale
from pprint import pprint
import re

from bs4 import BeautifulSoup
import requests as req

In [38]:
# função que identifica se o attributo href de qualquer elemento 
# da página corresponde ao link de uma partida
def filtra_links_partidas(href):
    return href and re.compile('linha$').search(href)

In [41]:
# identificando o número máximo de partidas por competição

numero_partidas = {
    'campeonato-brasileiro-serie-a': {},
    'copa-brasil-masculino': {}
}

for competicao in numero_partidas.keys():

    for temporada in range(2012, 2022):
        
        url_competicao = f'https://www.cbf.com.br/futebol-brasileiro/competicoes/{competicao}/{temporada}'
        res = req.get(url_competicao)
        soup = BeautifulSoup(res.text)

        ids_partida = []
        for a in soup.find_all(href=filtra_links_partidas):
            href = a['href']
            id_partida = href.split('/')[-1].split('?')[0]
            ids_partida.append(int(id_partida))

        numero_partidas[competicao][temporada] = max(ids_partida)

In [48]:
print(json.dumps(numero_partidas, indent=4))

{
    "campeonato-brasileiro-serie-a": {
        "2012": 380,
        "2013": 380,
        "2014": 380,
        "2015": 380,
        "2016": 380,
        "2017": 380,
        "2018": 380,
        "2019": 380,
        "2020": 380,
        "2021": 380
    },
    "copa-brasil-masculino": {
        "2012": 126,
        "2013": 172,
        "2014": 172,
        "2015": 172,
        "2016": 170,
        "2017": 120,
        "2018": 120,
        "2019": 120,
        "2020": 120,
        "2021": 122
    }
}


In [112]:
churros = {'campeonato-brasileiro-serie-a': {'2012': 3, '2013': 3, '2014': 3}, 'copa-brasil-masculino': {'2012': 3, '2013': 3, '2014': 3}}
print(json.dumps(churros, indent=4))

{
    "campeonato-brasileiro-serie-a": {
        "2012": 3,
        "2013": 3,
        "2014": 3
    },
    "copa-brasil-masculino": {
        "2012": 3,
        "2013": 3,
        "2014": 3
    }
}


In [161]:
def scrap_partida(url):
    res = req.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    dados = [competicao, temporada, id_partida]

    agenda = [span.text.strip() for span in soup.find_all('span', class_='text-2 p-r-20')][:3]
    dados.extend(agenda)

    mandante = re.split('\n+', soup.find('div', class_='time-left').text)[1:-1]
    dados.extend(mandante[:2])
    dados.append(sep.join(mandante[2:]))

    visitante = re.split('\n+', soup.find('div', class_='time-right').text)[1:-1]
    dados.extend(visitante[:2])
    dados.append(sep.join(visitante[2:]))

    gols_contra = set([p.text for p in soup.find_all('p', class_='color-red')])
    dados.append(sep.join(gols_contra))

    penaltis = [0, 0]
    try:
        disputa_penaltis = re.split('\n+', soup.find('div', class_='x center-block').text)
        if len(disputa_penaltis)>2:
            penaltis = disputa_penaltis[2:4]
    except AttributeError:
        penaltis = [0, 0]    
    dados.extend(penaltis)
    
    dados = [str(x) for x in dados]
    
    return dados

In [170]:
sep = ' |'
arquivo_resultados = 'dados/raw-scrap.csv'
erros = []

for competicao, temporadas in numero_partidas.items():
    print(competicao, end='')
    
    for temporada, max_partidas in temporadas.items():
        print(f'\n\ttemporada {temporada}:', end='')
        
        for id_partida in range(1, max_partidas+1):
            
            url = f'https://www.cbf.com.br/futebol-brasileiro/competicoes/{competicao}/{temporada}/{id_partida}'
            
            try:
                dados = scrap_partida(url)
                linha = ','.join(dados)
                linha += '\n'
                with open(arquivo_resultados, 'a', encoding='utf8') as f:
                    f.write(linha)
            except Exception as e:
                erro = (competicao, temporada, id_partida, str(e))
                erros.append(erro)
            finally:
                loader = int(id_partida/max_partidas*100)
                print('\r\ttemporada %s: partida %03d [%02d%%]' % (temporada, id_partida, loader), end='')
    
    print()

campeonato-brasileiro-serie-a
	temporada 2012: partida 380 [100%]
	temporada 2013: partida 380 [100%]
	temporada 2014: partida 380 [100%]
	temporada 2015: partida 380 [100%]
	temporada 2016: partida 380 [100%]
	temporada 2017: partida 380 [100%]
	temporada 2018: partida 380 [100%]
	temporada 2019: partida 380 [100%]
	temporada 2020: partida 380 [100%]
	temporada 2021: partida 380 [100%]
copa-brasil-masculino
	temporada 2012: partida 126 [100%]
	temporada 2013: partida 172 [100%]
	temporada 2014: partida 172 [100%]
	temporada 2015: partida 172 [100%]
	temporada 2016: partida 170 [100%]
	temporada 2017: partida 120 [100%]
	temporada 2018: partida 120 [100%]
	temporada 2019: partida 120 [100%]
	temporada 2020: partida 120 [100%]
	temporada 2021: partida 122 [100%]


In [171]:
erros

[]